In [ ]:
from ratdata import data_manager as dm, process, ingest
from pathlib import Path
import numpy as np
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
f_min = 2
f_max = 100
outfile = 'data/oof-%d-%d.txt' % (f_min, f_max)
with open(outfile, 'w') as f:
    f.write('filename,oof_exponent,stim\n')

all_recordings = dm.RecordingFile.select().count()
for i, r in enumerate(dm.RecordingFile.select()):
    full_filename = Path(r.dirname) / r.filename
    if dm.is_recording_rejected(r.filename):
        continue
    data = ingest.read_mce_matlab_file(str(full_filename))
    if dm.is_recording_sliced(r.filename):
        slice = r.slice.get()
        slice_start = int(slice.start / data.dt)
        slice_end = slice_start + int(slice.length / data.dt)
        x = np.mean(data.electrode_data[:, slice_start:slice_end], 0)
    else:
        x = np.mean(data.electrode_data, 0)
    fs = int(1/data.dt)
    ff, pxx = signal.welch(x, fs, nperseg=fs)
    m, b = process.fit_oof(ff, pxx, f_min, f_max)
    if r.stim.count() != 0 and r.stim.get().stim_type != 'nostim':
        stim = r.stim.get().stim_type
    else:
        stim = 'nostim'
    print('[%d/%d] %s %.2f %s' % (i+1, all_recordings, r.filename, m, stim))
    with open(outfile, 'a') as f:
        f.write('%s,%.2f,%s\n' % (r.filename, m, stim))
        

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/oof-2-100.txt', sep=',')

df.columns = ['filename', 'oof_exponent', 'stim']

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='stim', y='oof_exponent', data=df, palette='Dark2', boxprops=dict(alpha=.8))
sns.swarmplot(x='stim', y='oof_exponent', data=df, s=4, palette='Dark2')
plt.savefig('plots/oof/oof_exponent.png', bbox_inches='tight', facecolor='white')

In [ ]:
import scipy.stats as stats
from statannot import add_stat_annotation

In [ ]:
f_value, p_value = stats.f_oneway(df.loc[df['stim'] == 'continuous']['oof_exponent'], df.loc[df['stim'] == 'nostim']['oof_exponent'], df.loc[df['stim'] == 'on-off']['oof_exponent'], df.loc[df['stim'] == 'random']['oof_exponent'], df.loc[df['stim'] == 'proportional']['oof_exponent'], df.loc[df['stim'] == 'low']['oof_exponent'], df.loc[df['stim'] == 'low20']['oof_exponent'])
print(f_value, p_value)

In [ ]:
res = stats.tukey_hsd(df.loc[df['stim'] == 'nostim']['oof_exponent'], df.loc[df['stim'] == 'continuous']['oof_exponent'], df.loc[df['stim'] == 'on-off']['oof_exponent'], df.loc[df['stim'] == 'random']['oof_exponent'], df.loc[df['stim'] == 'proportional']['oof_exponent'], df.loc[df['stim'] == 'low']['oof_exponent'], df.loc[df['stim'] == 'low20']['oof_exponent'])

In [ ]:
labels = ['nostim', 'continuous', 'on-off', 'random', 'proportional', 'low', 'low20']
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='stim', y='oof_exponent', data=df, order=labels, palette='Dark2', boxprops=dict(alpha=.8))
sns.swarmplot(x='stim', y='oof_exponent', data=df, s=4, order=labels, palette='Dark2')
three_star_pairs = []
for i in range(len(labels)):
    for j in range(i + 1, len(labels)):
        if res.pvalue[i, j] < 1e-6:
            three_star_pairs.append((labels[i], labels[j]))
add_stat_annotation(ax, data=df, x='stim', y='oof_exponent', order=labels, box_pairs=three_star_pairs, text_format='star', test='Mann-Whitney')
plt.savefig('plots/oof/oof_exponents_stat.png', facecolor='white', bbox_inches='tight')